# Check input files
**Author: Jun Sasaki  Coded on 2024-12-26  Updated on 2024-12-29**<br>
Checking input netcdf files

In [ ]:
import xarray as xr
import os
#from xfvcom import Fvcom, FvcomPlotConfig, FvcomDataArray, FvcomPlotter
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_index_by_value(array, value):
    """
    指定された値のインデックスを返す関数

    Parameters:
        array (list or numpy.ndarray): 検索対象の配列（重複なし）
        value (int or float): 検索する値

    Returns:
        int: 配列内の値のインデックス（0から始まる）
    """
    try:
        # リストの場合
        return array.index(value)
    except AttributeError:
        # NumPy配列の場合
        index_array = np.where(array == value)[0]
        if index_array.size > 0:
            return int(index_array[0])  # 最初のインデックスを返す
        else:
            raise ValueError(f"{value} は配列内に存在しません。")
    except ValueError:
        raise ValueError(f"{value} は配列内に存在しません。")

In [ ]:
'''
def get_ncpath(indir, ncfile):
    indir = os.path.expanduser(indir)
    ncpath= f"{indir}/{ncfile}"
    if os.path.isfile(ncpath):
        return ncpath
    else:
        print(f"ERROR: File not found: {ncpath}")
        return None
'''

In [ ]:
indir = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
ncfile = "obc_dye2020.nc"

In [ ]:
#ncfile = "TokyoBay18_2020_wnd.nc"
#obc_dye2020 = Fvcom(dirpath=indir, ncfile=ncfile)
obc_dye2020 = FvcomDataLoader(indir, ncfile, time_tolerance=5)

In [ ]:
obc_dye2020.ds

In [ ]:
obc_nodes = obc_dye2020.ds.obc_nodes.values
print(f"obc_nodes={obc_nodes}")

In [ ]:
obc_node = 3150
obc_dye = obc_dye2020.ds['obc_dye']
obc_h = obc_dye2020.ds['obc_h']
obc_node_index = get_index_by_value(obc_nodes, obc_node)
time = obc_dye2020.ds['time']
siglay = obc_dye2020.ds['siglay'][:, obc_node_index]
dye_concentration = obc_dye[:, :, obc_node_index]
time_grid, siglay_grid = np.meshgrid(time, siglay)

In [ ]:
plt.figure(figsize=(10, 3))
plt.contourf(time_grid, siglay_grid, dye_concentration.T, cmap='viridis', levels=20)
plt.colorbar(label="Dye Concentration")
plt.xlabel("Time")
plt.ylabel("Sigma Layer Depth (siglay)")
plt.title(f"Dye Concentration at Open Boundary Node {obc_node}")
plt.xticks(rotation=45)
plt.tight_layout()
pngfile = f"dye_concentration_at_open_bound_node_{obc_node}.png"
plt.savefig(pngfile, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
selected_time = '2020-01-01'
selected_time = np.datetime64(selected_time)
dye_profile = obc_dye.sel(time=selected_time, method='nearest')
depth = siglay * obc_h  # 各ノードごとの水深を計算
depth = depth.transpose('siglay', 'nobc') 
nobc_indices = np.tile(obc_h.nobc.values, (len(siglay), 1)) 
nobc_numbers = np.tile(obc_nodes, (len(siglay),1))
siglay_grid = np.tile(siglay.values[:, None], (1, len(obc_nodes)))

In [ ]:
plt.close()
yaxis_siglay=False
plt.figure(figsize=(10, 3))
if yaxis_siglay:
    plt.contourf(
        nobc_indices, siglay_grid, dye_profile.values, levels=20, cmap='viridis'
    )
else:
    plt.contourf(
        nobc_indices, depth, dye_profile.values, levels=20, cmap='viridis'
    )
plt.colorbar(label="Dye Concentration")
plt.xlabel("Boundary Node Index")
plt.ylabel("Depth (m)")
plt.title(f"Dye Concentration Cross-Section on {selected_time}")
plt.gca().invert_yaxis()  # 水深を正の方向に（深くなるほど下へ）
#plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import holoviews as hv
import panel as pn
import hvplot.xarray
from datetime import datetime
hv.extension('bokeh')
pn.extension()

In [ ]:
ds = obc_dye2020.ds.copy()

In [ ]:
node_select = pn.widgets.Select(
    name='Node Index',
    options=list(range(ds.sizes['nobc'])),  # 0〜12
    value=0
)

def plot_time_siglay(node):
    """
    指定したノードの染料濃度を時系列とsiglayに基づいてヒートマップとしてプロットします。
    """
    # 指定したノードのデータを抽出（nobcを削除）
    dye_profile = ds['obc_dye'].isel(nobc=node, drop=True)  # (time, siglay)
    
    # プロットは 'time' vs 'siglay'
    heatmap = dye_profile.hvplot.quadmesh(
        x='time',
        y='siglay',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Node {node} Over Time",
        xlabel='Time',
        ylabel='Sigma Layer'
    )
    
    return heatmap

# 動的なプロットを作成
#interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# レイアウトの作成
#layout1 = pn.Column(
#    "# Dye Concentration Over Time",
#    node_select,
#    interactive_plot1
#)

In [ ]:
interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# レイアウトの作成
layout1 = pn.Column(
    "### Dye Concentration Over Time",
    node_select,
    interactive_plot1
)

In [ ]:
final_layout = pn.Row(layout1)

In [ ]:
final_layout

In [ ]:
# 時間選択用のウィジェット
time_options = [pd.to_datetime(t).strftime('%Y-%m-%d %H:%M') for t in ds['time'].values]

time_select = pn.widgets.DiscreteSlider(
    name='Time',
    options=time_options,  # 文字列形式のリスト
    value=time_options[0]
)

def plot_node_depth(time):
    """
    指定した時間の染料濃度をノードインデックスと水深に基づいてヒートマップとしてプロットします。
    """
    # 選択された時間を np.datetime64 に変換
    time_np = np.datetime64(time)
    
    # 指定した時間のデータを抽出
    dye_profile = ds['obc_dye'].sel(time=time_np)  # (siglay, nobc)
    
    # ノードインデックスを0〜12に設定
    node_indices = np.arange(ds.sizes['nobc'])
    dye_profile = dye_profile.assign_coords(node_index=('nobc', node_indices))
    
    # siglayとobc_hを取得
    # siglayは (siglay, nobc) だが、全ノードで同じと仮定
    siglay_1d = ds['siglay'].isel(nobc=0).values  # (30,)
    obc_h_values = ds['obc_h'].values  # (13,)
    
    # 水深を計算: depth = siglay * obc_h
    depth = siglay_1d[:, np.newaxis] * obc_h_values  # shape (30,13)
    
    # depthを座標として追加
    dye_profile = dye_profile.assign_coords(depth=(['siglay', 'nobc'], depth))

    max_depth = depth.max()
    min_depth = depth.min()
    
    # プロット
    heatmap = dye_profile.hvplot.quadmesh(
        x='node_index',
        y='depth',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Time {time} by Depth",
        xlabel='Node Index',
        ylabel='Depth (m)',
        ylim=(max_depth, min_depth)
    )

    return heatmap

# 動的なプロットを作成
interactive_plot2 = pn.bind(plot_node_depth, time=time_select)

# レイアウトの作成
layout2 = pn.Column(
    "# Dye Concentration by Depth",
    time_select,
    interactive_plot2
)


In [ ]:
final_layout = pn.Row(layout2)
final_layout

# Slice netcdf with time
To check netcdf files on PC, it is necessary to make their sizes much smaller; so slicing with time

In [ ]:
indir = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
ncfile = "TokyoBay18_2020_wnd.nc"
fvcom_wnd = FvcomDataLoader(dirpath=indir, ncfile=ncfile, time_tolerance=5)

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
output_path = f"sliced_{ncfile}"
fvcom_wnd.slice_by_time(start, end).to_netcdf(output_path)

In [ ]:
fvcom_wnd.ds

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
indir = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
ncfiles = ["TokyoBay2020final_tsobc.nc", "TokyoBay2020julian_obc.nc",
           "TokyoBay2020kisarazufinal_sewer.nc", "TokyoBay2020final_river.nc",
           "TokyoBay2020final_sewer.nc"]
for ncfile in ncfiles:
    output_path = f"sliced_{ncfile}"
    fvcom = FvcomDataLoader(dirpath=indir, ncfile=ncfile, time_tolerance=5)
    fvcom.slice_by_time(start, end).to_netcdf(output_path)